In [ ]:
from sklearn.metrics import accuracy_score
from tabpfn.scripts.decision_boundary import DecisionBoundaryDisplay
from tabpfn import TabPFNClassifier
import pandas as pd
import time
from tabpfn.scripts.decision_boundary import DecisionBoundaryDisplay

In [ ]:
import warnings
from ucimlrepo import fetch_ucirepo
warnings.filterwarnings('ignore')

In [ ]:
import import_ipynb
import numpy as np
import torch

In [ ]:
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
Xf = iris.data.features 
yf = iris.data.targets 
classes=yf['class'].unique().tolist()
perm = np.random.permutation(150).tolist()
Xf,yf=Xf.iloc[perm],yf.iloc[perm]

In [ ]:
yf.columns

In [ ]:
def encode_target(t,classes=classes):
    return classes.index(t)
yf['label']=yf['class'].apply(encode_target)

In [ ]:
train_n=100
X=torch.tensor(Xf.iloc[0:train_n].values, dtype=torch.float32)
y=torch.tensor(yf.iloc[0:train_n]['label'].values,dtype=torch.float32)
Xtest=torch.tensor(Xf.iloc[train_n:].values,dtype=torch.float32)
ytest=torch.tensor(yf[train_n:]['label'].values,dtype=torch.float32)

In [ ]:
classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=4)

In [ ]:
classifier.model

In [ ]:
X.shape

In [ ]:
start = time.time()
classifier.fit(X, y,overwrite_warning=True)
y_eval, p_eval = classifier.predict(Xtest, return_winning_probability=True)
prediction_time = time.time() - start

In [ ]:
print('Prediction time: ',prediction_time , 'Accuracy', accuracy_score(ytest, y_eval))

In [ ]:
p_eval

In [ ]:
y_eval_train, p_eval_train = classifier.predict(X, return_winning_probability=True)

In [ ]:
print('Prediction time: ',prediction_time , 'Accuracy', accuracy_score(y, y_eval_train))

In [ ]:
probs=classifier.predict_proba(Xtest)

In [ ]:
probs

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(ytest, y_eval)

In [ ]:
def gini(probs):
    # return [1-sum([p**p for p in pv]) for pv in probs]
    return [1-sum([p**2 for p in pv]) for pv in probs]


In [ ]:
gini(probs)

In [ ]:
# fetch dataset 
beans = fetch_ucirepo(id=850) 
# data (as pandas dataframes) 
Xf = beans.data.features 
yf = beans.data.targets 

In [ ]:
yf

In [ ]:
target='Class'
classes=yf[target].unique().tolist()
perm = np.random.permutation(Xf.shape[0]).tolist()
Xf,yf=Xf.iloc[perm],yf.iloc[perm]

In [ ]:
def encode_target(t,classes=classes):
    return classes.index(t)

In [ ]:
yf['label']=yf['Class'].apply(encode_target)

In [ ]:
Xf.shape

In [ ]:
train_n=700
X=torch.tensor(Xf.iloc[0:train_n].values, dtype=torch.float32)
y=torch.tensor(yf.iloc[0:train_n]['label'].values,dtype=torch.float32)
Xtest=torch.tensor(Xf.iloc[train_n:].values,dtype=torch.float32)
ytest=torch.tensor(yf[train_n:]['label'].values,dtype=torch.float32)

In [ ]:
classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=4)

In [ ]:
start = time.time()
classifier.fit(X, y,overwrite_warning=True)
y_eval, p_eval = classifier.predict(Xtest, return_winning_probability=True)
prediction_time = time.time() - start

In [ ]:
print('Prediction time: ',prediction_time , 'Accuracy', accuracy_score(ytest, y_eval))

In [ ]:
confusion_matrix(ytest, y_eval)

## Calibration

In [ ]:
from torchmetrics.functional import calibration_error

In [ ]:
from sklearn.calibration import CalibrationDisplay

In [ ]:
probs=torch.tensor(classifier.predict_proba(Xtest))

In [ ]:
probs

In [ ]:
def binarize(target,c):
    return np.array([1 if t==c else 0 for t in target])

In [ ]:
binarize(ytest,1)

In [ ]:
probs[:,1]

In [ ]:
CalibrationDisplay.from_predictions(y_true=binarize(ytest,1),y_prob=probs[:,1])

In [ ]:
probs_train=torch.tensor(classifier.predict_proba(X)).softmax(dim=1)

In [ ]:
CalibrationDisplay.from_predictions(y_true=binarize(y,1),y_prob=probs_train.softmax(dim=1)[:,1])

In [ ]:
from temperature_scaling import ModelWithTemperature

In [ ]:
from functools import partial

In [ ]:
import torch.nn as nn

In [ ]:
class WrappedModel(nn.Module):
    def __init__(self,tabpfnmodel,label_index=None):
        super().__init__()
        self.classifier=tabpfnmodel
        self.label_index=label_index
    def __call__(self,X):
        probs=self.classifier.predict_proba(X)
        if self.label_index is None: return torch.tensor(probs)
        else: return torch.tensor(probs[:,self.label_index])

In [ ]:
tabmodel=WrappedModel(classifier,label_index=1)

In [ ]:
tabmodel(Xtest)

In [ ]:
model=ModelWithTemperature(tabmodel)

In [ ]:
from utils import MyDS

In [ ]:
loader=torch.utils.data.DataLoader(MyDS(Xf.iloc[train_n:].values,yf[train_n:]['label'].values))

In [ ]:
model=model.set_temperature(loader)

In [ ]:
probs_cal=model(Xtest).detach()

In [ ]:
CalibrationDisplay.from_predictions(y_true=binarize(ytest,1),y_prob=probs_cal.softmax(dim=1)[:,1])

In [ ]:
CalibrationDisplay.from_predictions(y_true=binarize(ytest,0),y_prob=probs_cal.softmax(dim=1)[:,0],n_bins=20)

In [ ]:
calibration_error(preds=probs_cal,target=torch.tensor(ytest),task='multiclass',num_classes=3)

In [ ]:
calibration_error(preds=torch.tensor(probs),target=torch.tensor(ytest),task='multiclass',num_classes=3)

In [ ]:
from temperature_scaling import _ECELoss

In [ ]:
ece=_ECELoss(n_bins=10)

In [ ]:
ece.forward(probs_cal,ytest)

In [ ]:
ece.forward(probs,ytest)